In [1]:
import pandas as pd
from itertools import zip_longest

### Load Datasets

In [2]:
air_quality_dat = pd.read_csv("data/CPSC368 DATA - Air Quality.csv")
ghg_emissions_dat_wide = pd.read_csv("data/CPSC368 DATA - GHG Emissions.csv")
global_climate_index_dat = pd.read_csv("data/CPSC368 DATA - Global Climate Index.csv")
paris_agreement_dat = pd.read_csv("data/CPSC368 DATA - Paris Agreement.csv")

### Clean Datasets

In [3]:
# Clean data

# Make country names the same
global_climate_index_dat["Country"] = global_climate_index_dat["Country"].str.replace("-"," ")

# Remove null/non-country rows at the end of the datasets
ghg_emissions_dat_wide = ghg_emissions_dat_wide[:-4]

# Make year a column in greenhouse gas emissions data
ghg_emissions_dat = pd.melt(ghg_emissions_dat_wide, id_vars='Country', value_vars=['1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022'], var_name = "Year", value_name = "Emissions")

# Change a country name to match the others
def alter_country_name(dataset, alter_dict, country_column_name):
    '''change country names in a dataset so it matches the values in other datasets
       example-- changing Brunei to Brunei Darussalam in one dataset so all datasets refer to Brunei in the same way

       INPUTS:
       dataset: dataset for which we are altering the country name
       change_dict: a dictionary where the key is country name we want to alter and the value is what we are changing to
       country_column_name: the column name of the column that contains countries

       RETURNS:
       altered dataset
       '''
    for original, changed in alter_dict.items():
        dataset.loc[dataset[country_column_name] == original, country_column_name] = changed
    return dataset


ghg_emissions_dat = alter_country_name(ghg_emissions_dat, {'Brunei':"Brunei Darussalam",
                                                           "Czechia":"Czech Republic",
                                                           'Côte d’Ivoire':"Côte d'Ivoire",
                                                           "Iran":"Iran (Islamic Republic of)",
                                                           "Laos":"Lao People's Democratic Republic",
                                                           "South Korea":'Republic of Korea',
                                                           "The Gambia":"Gambia"},
                                       "Country")
paris_agreement_dat = alter_country_name(paris_agreement_dat, {'Bolivia (Plurinational State of)':'Bolivia',
                                                               "Denmark 1":"Denmark",
                                                               "Micronesia (Federated States of)":"Micronesia",
                                                               "Netherlands (Kingdom of the) 2":"Netherlands",
                                                               "New Zealand 3":"New Zealand",
                                                               "Republic of Moldova":"Moldova",
                                                               "Yemen":'Republic of Yemen',
                                                               'St. Kitts and Nevis':'St.Kitts and Nevis',
                                                               'St. Lucia':'St.Lucia',
                                                               'St. Vincent and the Grenadines':'St.Vincent and the Grenadines',
                                                               'United Kingdom of Great Britain and Northern Ireland 4, 5, 6':'United Kingdom',
                                                               "United States of America 7":"United States of America",
                                                               "Viet Nam":"Vietnam",
                                                               "Venezuela (Bolivarian Republic of)":'Venezuela',
                                                               "United Republic of Tanzania":"Tanzania"},
                                         "Participant")
air_quality_dat = alter_country_name(air_quality_dat, {'Bolivia (Plurinational State of)':'Bolivia',
                                                       "Czechia":"Czech Republic",
                                                       "Viet Nam":"Vietnam",
                                                       "Venezuela (Bolivarian Republic of)":'Venezuela',
                                                       "United Republic of Tanzania":"Tanzania"},
                                     "WHO Country Name") 
global_climate_index_dat = alter_country_name(global_climate_index_dat, {'Republic of Congo':'Congo',
                                                                         "Islamic Republic of Iran":"Iran (Islamic Republic of)",
                                                                         'Islamic Republic of Afghanistan':"Afghanistan",
                                                                         "Kyrgyz Republic":"Kyrgyzstan",
                                                                         "Korea, Republic of":'Republic of Korea',
                                                                         "Russia":"Russian Federation",
                                                                         'Slovak Republic':'Slovakia',
                                                                         'United States':'United States of America',
                                                                         "The Gambia":"Gambia"},
                                              "Country") 

### Create Insert Statements

In [4]:
# Clear conents of insert statement text file for clean state
open('data/insert_statements.txt','w').close()

In [5]:
# Find out which countries are not in the air-quality dataset
aq_countries = set(air_quality_dat["WHO Country Name"].unique())
ghg_emissions_countries = set(ghg_emissions_dat["Country"].unique())
gci_countries = set(global_climate_index_dat["Country"].unique())
pa_countries = set(paris_agreement_dat["Participant"].unique())
country_names_not_in_aq_countries = aq_countries.union(ghg_emissions_countries).union(gci_countries).union(pa_countries) - aq_countries

In [6]:
# Looking for inconsistencies in how the countries are represented textually
aq = sorted(aq_countries)
ghg = sorted(ghg_emissions_countries)
gci = sorted(gci_countries)
pa = sorted(pa_countries)

for c1, c2, c3, c4 in zip_longest(aq, ghg, gci, pa, fillvalue = ''):
    print(f'{c1:<35} {c2:<35} {c3:<35} {c4}')

Afghanistan                         Afghanistan                         Afghanistan                         Afghanistan
Albania                             Albania                             Albania                             Albania
Algeria                             Algeria                             Algeria                             Algeria
Andorra                             Angola                              Angola                              Andorra
Argentina                           Anguilla                            Antigua and Barbuda                 Angola
Australia                           Antigua and Barbuda                 Argentina                           Antigua and Barbuda
Austria                             Argentina                           Armenia                             Argentina
Bahamas                             Armenia                             Australia                           Armenia
Bahrain                             Aruba              

In [7]:
# Create Country Table Insert Statements

# open insert statements file
insert_statements=open('data/insert_statements.txt','w')

# create table with country names and attributes from the air-quality dataset
aq_countries_df = air_quality_dat[["WHO Country Name","WHO Region","ISO3"]].drop_duplicates()

# create insert statements for countries in the air-quality dataset
for index, row in aq_countries_df.iterrows():
    insert_statements.write(f'INSERT INTO Country(Country_Name, WHO_Region, ISO)\n VALUES ({row["WHO Country Name"]},{row["WHO Region"]},{row["ISO3"]}) \n')

# create insert statements for countries NOT in the air-quality dataset
for leftover_country in country_names_not_in_aq_countries:
    insert_statements.write(f'INSERT INTO Country(Country_Name, WHO_Region, ISO)\n VALUES ({leftover_country},NULL,NULL) \n')

insert_statements.write(f'\n \n \n \n')
insert_statements.close()

In [8]:
# Create City Table Insert Statements

# open insert statements file
insert_statements=open('data/insert_statements.txt','a')

# create table with country names and city names from the air-quality dataset
aq_cities_df = air_quality_dat[["WHO Country Name","City or Locality"]].drop_duplicates()

# create insert statements for countries in the air-quality dataset
for index, row in aq_cities_df.iterrows():
    insert_statements.write(f'INSERT INTO City(City_Name,Country_Name)\n VALUES ({row["City or Locality"]},{row["WHO Country Name"]}) \n')

insert_statements.write(f'\n \n \n \n \n')
insert_statements.close()

In [9]:
# Create Air_Quality_Measures Table Insert Statements

# open insert statements file
insert_statements=open('data/insert_statements.txt','a')

# create insert statements for countries in the air-quality dataset
for index, row in air_quality_dat.iterrows():
    insert_statements.write(f'INSERT INTO Air_Quality_Measures(ID, Year, City_Name, Country_Name, NO2, PM10, PM2.5, NO2_temporal_coverage, PM2.5_temporal_coverage, PM10_temporal_coverage)\n VALUES ({index}, {row["Measurement Year"]}, {row["City or Locality"]}, {row["WHO Country Name"]}, {row["NO2 (μg/m3)"]}, {row["PM10 (μg/m3)"]}, {row["PM2.5 (μg/m3)"]}, {row["NO2 temporal coverage (%)"]}, {row["PM25 temporal coverage (%)"]}, {row["PM10 temporal coverage (%)"]}) \n')

insert_statements.write(f'\n \n \n \n \n')
insert_statements.close()

In [10]:
# Create Greenhouse_Gasses Table Insert Statements

insert_statements=open('data/insert_statements.txt','a')

# create insert statements for countries in the air-quality dataset
for index, row in ghg_emissions_dat.iterrows():
    insert_statements.write(f'INSERT INTO Greenhouse_Gasses(Year, Country_Name, Per_Capita_GHG) \n VALUES ({row["Year"]},{row["Country"]},{row["Emissions"]}) \n')

insert_statements.write(f'\n \n \n \n \n')
insert_statements.close()

In [11]:
# Create Paris_Agreement_Member Table Insert Statements

# Obtain countries not in Paris agreement
country_names_not_in_pa_countries = aq_countries.union(ghg_emissions_countries).union(gci_countries).union(pa_countries) - pa_countries

# open insert statements file
insert_statements=open('data/insert_statements.txt','a')

# create table with country names and attributes from the paris agreement dataset
pa_countries_df = paris_agreement_dat[["Participant","Ratification, Acceptance(A), Approval(AA), Accession(a)"]].drop_duplicates()

# create insert statements for countries in the paris agreement dataset
for index, row in pa_countries_df.iterrows():
    insert_statements.write(f'INSERT INTO Paris_Agreement_Member(Country_Name, Date_Joined)\n VALUES ({row["Participant"]},{row["Ratification, Acceptance(A), Approval(AA), Accession(a)"]}) \n')

insert_statements.write(f'\n \n \n \n')
insert_statements.close()

In [12]:
# Create Climate_Risk_Index Table Insert Statements

# open insert statements file
insert_statements=open('data/insert_statements.txt','a')

# create insert statements for countries in the climate risk index dataset
for index, row in global_climate_index_dat.iterrows():
    insert_statements.write(f'INSERT INTO Climate_Risk_Index(ID, Country_Name, CRI_Rank, CRI_Score, Fatalities_2018_Rank, Fatalities_Per_100000_Rank, US_Million_Losses_Rank, GDP_Losses_Rank) \n VALUES ({row["ID"]},{row["Country"]},{row["CRI Rank"]},{row["CRI score"]},{row["Fatalities in 2018 (Rank)"]},{row["Fatalities per 100 000 inhabitants (Rank)"]},{row["Losses in million US$ (PPP) (Rank)"]},{row["Losses per unit GDP in % (Rank)"]}) \n')

insert_statements.write(f'\n \n \n \n \n')
insert_statements.close()